# 3、梯度下降法的向量化、数据标准化对梯度下降的重要作用

In [1]:
import numpy as np
from sklearn import datasets

boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [2]:
from playML.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [3]:
from playML.LinearRegression import LinearRegression

lin_reg1 = LinearRegression()
%time lin_reg1.fit_normal(X_train, y_train)
lin_reg1.score(X_test, y_test)

CPU times: user 807 µs, sys: 446 µs, total: 1.25 ms
Wall time: 643 µs


0.8129802602658576

使用梯度下降法

In [4]:
lin_reg2 = LinearRegression()
lin_reg2.fit_gd(X_train, y_train)

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/liwei/Downloads/root_notebook/刘宇波老师《机器学习》课程笔记/第 6 章 梯度下降法/playML/LinearRegression.py:35: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(X_b)
/Users/liwei/Downloads/root_notebook/刘宇波老师《机器学习》课程笔记/第 6 章 梯度下降法/playML/LinearRegression.py:51: RuntimeWarning: invalid value encountered in double_scalars
  if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):


LinearRegression()

In [5]:
lin_reg2.coef_

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

可以发现，回归系数没有计算出来，所以说，我们的梯度下降法是有问题的。

我们首先看一下训练数据集。数量级有相差。

In [6]:
X_train[:5, :]

array([[1.42362e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.93000e-01,
        6.34300e+00, 1.00000e+02, 1.57410e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.96900e+02, 2.03200e+01],
       [3.67822e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 7.70000e-01,
        5.36200e+00, 9.62000e+01, 2.10360e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.80790e+02, 1.01900e+01],
       [1.04690e-01, 4.00000e+01, 6.41000e+00, 1.00000e+00, 4.47000e-01,
        7.26700e+00, 4.90000e+01, 4.78720e+00, 4.00000e+00, 2.54000e+02,
        1.76000e+01, 3.89250e+02, 6.05000e+00],
       [1.15172e+00, 0.00000e+00, 8.14000e+00, 0.00000e+00, 5.38000e-01,
        5.70100e+00, 9.50000e+01, 3.78720e+00, 4.00000e+00, 3.07000e+02,
        2.10000e+01, 3.58770e+02, 1.83500e+01],
       [6.58800e-02, 0.00000e+00, 2.46000e+00, 0.00000e+00, 4.88000e-01,
        7.76500e+00, 8.33000e+01, 2.74100e+00, 3.00000e+00, 1.93000e+02,
        1.78000e+01, 3.95560e+02, 7.56000e+00]])

每一个数据对应的数据的规模是不同的，梯度下降法不收敛。所以，我们要调整 eta 和 n_iters 。

我们先调整学习率，让梯度减慢一些。

In [7]:
lin_reg2 = LinearRegression()
lin_reg2.fit_gd(X_train, y_train, eta=1e-6)

LinearRegression()

In [8]:
lin_reg2.score(X_test, y_test)

0.27556634853389195

效果很不好，那是因为梯度下降法最后没有走到极值点。

运行下面的这段代码太耗时了，循环次数增加了一百倍。

In [9]:
lin_reg2 = LinearRegression()
%time lin_reg2.fit_gd(X_train, y_train, eta=1e-6, n_iters=1e6)

CPU times: user 38.1 s, sys: 105 ms, total: 38.2 s
Wall time: 38.4 s


LinearRegression()

In [10]:
lin_reg2.score(X_test, y_test)

0.7541852353980762

当然最好的方法是，对数据进行标准化（归一化）。

## 使用梯度下降法前进行数据的归一化

In [11]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
X_train_standard = standardScaler.transform(X_train)

In [13]:
lin_reg3 = LinearRegression()
%time lin_reg3.fit_gd(X_train_standard, y_train)

CPU times: user 317 ms, sys: 8.92 ms, total: 326 ms
Wall time: 346 ms


LinearRegression()

**测试数据集使用训练数据集的期望和方差进行标准化。**  
**测试数据集使用训练数据集的期望和方差进行标准化。**  
**测试数据集使用训练数据集的期望和方差进行标准化。**

In [14]:
X_test_standard = standardScaler.transform(X_test)

In [15]:
lin_reg3.score(X_test_standard, y_test)

0.8129880620122235

在矩阵较大的时候，正规方程的方法计算量大。  
但是梯度下降法也有问题，当样本数量较大的时候，梯度下降法的计算量很大。

梯度下降法的优势。

In [16]:
m = 1000
n = 5000

big_X = np.random.normal(size=(m, n))
true_theta = np.random.uniform(0.0, 100.0, size=n + 1)
big_y = big_X.dot(true_theta[1:]) + true_theta[0] + \
    np.random.normal(0, 10, size=m)

In [17]:
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y)

CPU times: user 15.6 s, sys: 985 ms, total: 16.6 s
Wall time: 5.32 s


LinearRegression()

In [18]:
big_reg2 = LinearRegression()
%time big_reg1.fit_gd(big_X, big_y)

CPU times: user 8.67 s, sys: 300 ms, total: 8.97 s
Wall time: 2.3 s


LinearRegression()